論文  
https://arxiv.org/abs/2212.00490<br>
<br>
GitHub<br>
https://github.com/wyhuai/DDNM<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/DDNM_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/wyhuai/DDNM.git

# Commits on Dec 11, 2022
%cd /content/DDNM
!git checkout f157d760710169c9ba22a830caac9035ade71b5f

## ライブラリのインストール

In [ ]:
%cd /content/DDNM

!pip install blobfile==2.0.0 tqdm==4.61.1 pyYaml==6.0 pillow==7.1.2


## ライブラリのインポート

In [ ]:
import glob
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/DDNM

# Create dir
!mkdir -p DDNM/exp/logs/celeba DDNM/exp/logs/imagenet

# Download model
!wget -c https://image-editing-test-12345.s3-us-west-2.amazonaws.com/checkpoints/celeba_hq.ckpt \
      -O DDNM/exp/logs/celeba/celeba_hq.ckpt
!wget -c https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt \
      -O DDNM/exp/logs/imagenet/256x256_diffusion_uncond.pt

# 4x Super Resolution

In [ ]:
# base path
exp = './exp'
# input dir {exp}/datasets/{in_dir}
in_dir = 'celeba_hq'
# output dir
out_dir = 'sr_results' # {exp}/image_sample/{out_dir}
# tasks
tasks = 'sr_averagepooling' #@param ['cs_walshhadamard', 'cs_blockbased', 'inpainting', 'denoising', 'deblur_uni', 'deblur_gauss', 'deblur_aniso', 'sr_averagepooling', 'sr_bicubic', 'colorization', 'mask_color_sr']
# config path configs/{config_path}
config_path = 'celeba_hq.yml'

In [ ]:
%cd /content/DDNM

!python main.py \
  --ni --simplified \
  --config {config_path} \
  --path_y {in_dir} \
  --eta 0.85 \
  --deg {tasks} \
  --deg_scale 4.0 --sigma_y 0 \
  --exp {exp} \
  -i {out_dir}

In [ ]:
input_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/Apy/Apy_*.png')
input_imgs.sort()
output_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/*.png')
output_imgs.sort()

In [ ]:
for i, o in zip(input_imgs, output_imgs):
  in_img = Image.open(i)
  out_img = Image.open(o)

  fig = plt.figure(num=None, figsize=(12, 5))

  ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
  plt.imshow(np.asarray(in_img))
  ax.set_title('input')

  ax = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
  plt.imshow(np.asarray(out_img))
  ax.set_title('4x output')


# Old photo restoration

In [ ]:
# base path
exp = './exp'
# input dir {exp}/datasets/{in_dir}
in_dir = 'oldphoto'
# output dir
out_dir = 'opr_results' # {exp}/image_sample/{out_dir}
# tasks
tasks = 'mask_color_sr' #@param ['cs_walshhadamard', 'cs_blockbased', 'inpainting', 'denoising', 'deblur_uni', 'deblur_gauss', 'deblur_aniso', 'sr_averagepooling', 'sr_bicubic', 'colorization', 'mask_color_sr']
# config path configs/{config_path}
config_path = 'oldphoto.yml'

In [ ]:
%cd /content/DDNM

!python main.py \
  --ni --simplified \
  --config {config_path} \
  --path_y {in_dir} \
  --eta 0.85 \
  --deg {tasks} \
  --deg_scale 2.0 --sigma_y 0.02 \
  --exp {exp} \
  -i {out_dir}

In [ ]:
input_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/Apy/Apy_*.png')
input_imgs.sort()
output_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/*.png')
output_imgs.sort()

for i, o in zip(input_imgs, output_imgs):
  in_img = Image.open(i)
  out_img = Image.open(o)

  fig = plt.figure(num=None, figsize=(12, 5))

  ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
  plt.imshow(np.asarray(in_img))
  ax.set_title('input')

  ax = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
  plt.imshow(np.asarray(out_img))
  ax.set_title('old photo restoration output')

# Colorization

In [ ]:
%cd /content/DDNM

# defaultのテスト画像の名前変更
!mv ./exp/datasets/imagenet/imagenet ./exp/datasets/imagenet/org_imagenet
!mv ./exp/imagenet_val_1k.txt ./exp/org_imagenet_val_1k.txt 

!mkdir -p ./exp/datasets/imagenet/imagenet
!touch ./exp/imagenet_val_1k.txt
!echo "test01.jpg 0" >> /content/DDNM/exp/imagenet_val_1k.txt

!wget -c https://i0.wp.com/helenbushe.com/wp-content/uploads/2016/06/12542638003_6cbe8fabc5_b-685x1024.jpg \
      -O ./exp/datasets/imagenet/imagenet/test01.jpg

In [ ]:
# base path
exp = './exp'
# input dir {exp}/datasets/{in_dir}
in_dir = 'imagenet'
# output dir
out_dir = 'co_results' # {exp}/image_sample/{out_dir}
# tasks
tasks = 'colorization' #@param ['cs_walshhadamard', 'cs_blockbased', 'inpainting', 'denoising', 'deblur_uni', 'deblur_gauss', 'deblur_aniso', 'sr_averagepooling', 'sr_bicubic', 'colorization', 'mask_color_sr']
# config path configs/{config_path}
config_path = 'imagenet_256.yml'

In [ ]:
%cd /content/DDNM

!python main.py \
  --ni --simplified \
  --config {config_path} \
  --path_y {in_dir} \
  --eta 0.85 \
  --deg {tasks} \
  --sigma_y 0. \
  --exp {exp} \
  -i {out_dir}

In [ ]:
input_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/Apy/Apy_*.png')
input_imgs.sort()
output_imgs = glob.glob(f'{exp}/image_samples/{out_dir}/*.png')
output_imgs.sort()

for i, o in zip(input_imgs, output_imgs):
  in_img = Image.open(i)
  out_img = Image.open(o)

  fig = plt.figure(num=None, figsize=(12, 5))

  ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
  plt.imshow(np.asarray(in_img))
  ax.set_title('input')

  ax = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
  plt.imshow(np.asarray(out_img))
  ax.set_title('colorization output')